<a href="https://colab.research.google.com/github/Anpchernova/My_EDA/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%80%D0%B5%D0%B7%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D1%82%D0%BE%D0%B2_AB_%D1%82%D0%B5%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Вступление**

Компания (e-commerce) разработала новую веб-страницу, чтобы попытаться увеличить число платящих пользователей своего продукта. Они провели AB тест: контрольной (control) группе они показывали старую страницу веб-сайта, а тестовой (treatment) - новую. По результатам данного AB теста я сделаю вывод: следует ли компании внедрить эту новую страницу, сохранить старую страницу или, возможно, продлить эксперимент, чтобы принять решение.

### **Этапы AB теста:**
1. Формулирование гипотезы:
  *Если мы изменим дизайн веб-страницы, то это увеличит число платящих пользователей.*

2. Проверка допущений:
- Допущение о нормальности;
- Гомогенность (однородность) дисперсии.
3. Проведение эксперимента:
- Если допущения из шага 2 выполнены, то проводим параметрический тест с t-критерием;
- В противном случае непараметрический тест Манна Уитни с u-критерием.
4. Результаты интерпретируются на основе значения p-value.

### **Подготовка перед AB тестом**

1. Формулирование гипотез:

**H0**: Статистически значимой разницы между старой страницей и новой страницей нет.

**H1**: Существует статистически значимая разница между старой страницей и новой страницей.

2. Проверка допущений:

**Допущение о нормальности:**

- H0: Предполагается, что распределение нормальное.

- H1: Не предполагается, что распределение нормальное.

Если значение p-value меньше 0.05, то оно считается значимым, и будет использован непараметрический тест (u-критерий Манна-Уитни). В противном случае параметрический тест (t-критерий).

**Гомогенность дисперсии:**
- H0: Дисперсии однородны.
- H1: Дисперсии неоднородны.

### **Заключение**

Гипотеза будет проверена на основе значения p-value, полученного в результате параметрического/непараметрического теста, который мы проведем. Этот результат ответит на вопрос: есть ли существенная разница между новой и старой страницей?

### *Проведение эксперимента...*
---


Далее импортируем набор данных для анализа результатов AB-теста.
Был взят от [сюда](https://www.kaggle.com/datasets/putdejudomthai/ecommerce-ab-testing-2022-dataset1).


---

### **Немного о самом наборе данных:**
- user_id: уникальные id пользователей
- timestamp: отметка времени
- group: control или treatment
- landing_page: old_page (старый дизайн страницы) или new_page (новый дизайн страницы)
- converted: статус регистрации после просмотра страницы (0-1)


In [ ]:
import pandas as pd
import statsmodels.stats.api as sms
from scipy.stats import shapiro, levene, mannwhitneyu

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ab = pd.read_csv('/content/ab_data.csv')

### **Разведочный анализ данных (EDA)**

In [ ]:
ab.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [ ]:
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [ ]:
ab.apply(lambda x: x.nunique())

user_id         290585
timestamp        35993
group                2
landing_page         2
converted            2
dtype: int64

In [ ]:
ab.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [ ]:
print(ab.shape)
ab = ab.drop_duplicates(subset= 'user_id', keep= False)
print(ab.shape)

(294480, 5)
(286690, 5)


In [ ]:
ab.groupby(['group','landing_page']).agg({'landing_page': lambda x: x.value_counts()})

,,landing_page
group,landing_page,
control,old_page,143293
treatment,new_page,143397


In [ ]:
ab.groupby(['group','landing_page']).agg({'converted': 'mean'})

,,converted
group,landing_page,
control,old_page,0.120173
treatment,new_page,0.118726


In [ ]:
pd.DataFrame(ab.loc[:,'landing_page'].value_counts(normalize = True) * 100)

,landing_page
new_page,50.018138
old_page,49.981862


In [ ]:
ab[((ab['group'] == 'control') & (ab['landing_page'] == 'new_page')) |((ab['group'] == 'treatment') & (ab['landing_page'] == 'old_page')) ]

,user_id,timestamp,group,landing_page,converted


Control group = new page

Treatment group = old page

### **Проведение эксперимента (анализ данных AB теста)**

**Допущение о нормальности:**

- H0: Предполагается, что распределение нормальное.

- H1: Не предполагается, что распределение нормальное.

Если значение p-value меньше 0.05, то оно считается значимым, и будет использован непараметрический тест (u-критерий Манна-Уитни). В противном случае параметрический тест (t-критерий).

In [ ]:
test_stat, pvalue = shapiro(ab.loc[ab["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.3792334198951721


In [ ]:
test_stat, pvalue = shapiro(ab.loc[ab["landing_page"] == "new_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.37685757875442505




---


**Вывод:** p-value < 0.05, поэтому предположение о нормальности не предполагается. Мы будем использовать непараметрический тест (u-критерий Манна Уитни).

---



**Гомогенность дисперсии:**

- H0: Дисперсии однородны.
- H1: Дисперсии неоднородны.

In [ ]:
test_stat, pvalue = levene(ab.loc[ab["landing_page"] == "new_page", "converted"],
                           ab.loc[ab["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.2322897281547632
test_stat: 1.4267917566652295


**Вывод:** Дисперсии однородны.

### **Тестирование гипотез**
Непараметрический тест с u-критерием Манна Уитни.

- H0: Статистически значимой разницы между старой страницей и новой страницей нет.

- H1: Существует статистически значимая разница между старой страницей и новой страницей.

In [ ]:
test_stat, pvalue = mannwhitneyu(ab.loc[ab["landing_page"] == "new_page", "converted"],
                                 ab.loc[ab["landing_page"] == "old_page", "converted"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 10259026653.0000, p-value = 0.2323


**Вывод:** p-value(0.2323) > 0.05. Следовательно, мы не можем отклонить нулевую гипотезу. Статистически значимой разницы между новой и старой страницы нет.

### **Основной вывод по итогам AB теста:**
p-value(0.2323) > 0.05. Следовательно, мы не можем отклонить нулевую гипотезу. Статистически значимой разницы между новой и старой страницей нет.